# The Path to Empirical Orthogonal Function Analyses

Today we're going to continue to set the stage to use empirical orthogonal function (EOF) analysis (or principal components analysis) on our spatiotemporal data.  Recall the we anticipate that our data will be linearly dependent - that some of our time series can be expressed as linear combinations of others.  They will covary (have a non-trivial covariance), they will be correlated (have a significant correlation coefficient), and they will not be orthogonal (they will have a non-zero dot product).  As we've said, this both provides challenges (mathematically and statistically) but also potentially reveals something about the larger-scale and perhaps causal nature of our data.  EOF analysis and related techniques will allow us to both handle the challenge and leverage the advantages of our data. 

Before we get started, let's import commonly used packages as normal:

In [247]:
import numpy as np

## Special Matrices

Let's revisit some of the special matrices we learned about in linear algebra.  The first is the _**square matrices**_, which is simply a matrix with the same number of rows and columns.  Another (related) special matrix is the _**symmetrical matrices**_, where the following holds true:

$
\mathbf{A} = \mathbf{A}^T 
$

Or, a symmetrical matrix is a matrix that is equal to its transpose.  It is easy to see why a symmetrical matrix must also be a square matrix (how could a matrix with different numbers of rows and columns be equal to its transpose, which swaps columns for rows?).  One of the other special matrices we learned about -- the identity matrix -- is a square symmetric matrix with 1 on the diagonal and 0 everywhere else:


In [248]:
I = np.identity(4) # can also use np.eye(4)
print(I)
I == I.T # all entries in this equality comparison are true


[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

Remeber you can get the diagonal elements of a matrix using `np.diag`, `np.diagonal`, or `.diagonal()`, and you can get the sum of the diagonal elements (called the **trace**) using `np.trace` or `.trace()`:

In [249]:
# various ways of getting the diagonal elements
print(np.diag(I))
print(np.diagonal(I))
print(I.diagonal())

# getting the trace, the sum of the diagonal elements:
print(np.trace(I))
print(I.trace())


[1. 1. 1. 1.]
[1. 1. 1. 1.]
[1. 1. 1. 1.]
4.0
4.0


## Compatible matrices and matrix multiplication

By now you're probably quite tired of hearing me talk about matrix multiplication!  But being comfortable with it is key to building your intuition and ability to do EOF analyses using basic matrix operations.  Let's briefly review some of the most important points.

We can multiply matrices so long as the number of columns in the first matrix is the same as the number of rows in the second matrix.  Or, in otherwords, their _**inner dimensions**_ are the same.  In this case we can calculate the dot or inner product of two matrices or vectors.  Let's look at a simple example:

$
\mathbf{A} = [2\: {-1}\: 6\: 3]
$

Here, **A** is a vector with 1 row and 4 columns, or a 1x4 matrix (note that Python allows you to have 1-dimensional arrays, so a Python element can simply have 4 elements and not necessarily be row-vector or column-vector).  If we find the product:

$
\mathbf{A}\mathbf{A}^T
$

We can see that this would be the multiplication of a 1x4 vector with a 4x1 vector.  Since the inner dimensions are both 4, we can calculate the dot product, and since the outer dimensions are 1 and 1, we expect the outcome of this dot product to have a dimension of 1x1 -- or, a scalar:  

In [250]:
A = np.array([2, -1, 6, 3]);
print(A.shape) # Python allows one dimensional arrays(!), so to demonstrate what I'm talking about here, I ...

# Add another dimension to A so that it is truly (1,4), not (4,), so that np.dot will work as expected
A = A[np.newaxis,:]
print(A.shape) # A is now 2-dimensional, with 1 row and 4 columns

np.dot(A,A.T)


(4,)
(1, 4)


array([[50]])

Indeed, the dot product is a scalar equal to 50, which comes from (2x2) + (-1x-1) + (6x6) + (3x3) = 4 + 1 + 26 + 9 = 50.

What if we did it the otherway, though?  Remember, in matrix algebra multiplication is not commutative.  What if we calculate the dot product of:

$
\mathbf{A}^T\mathbf{A}
$

Now we would have a 4x1 matrix and a 1x4 matrix -- with the inner dimension of 1, the matrices are compatiable -- they can be multiplied -- but now we see that the outer dimensions lead us to expect a 4x4 matrix!  

In [251]:
B = np.dot(A.T,A)
print(B)

[[ 4 -2 12  6]
 [-2  1 -6 -3]
 [12 -6 36 18]
 [ 6 -3 18  9]]


What are these entries?  Take a moment and realize that (1) each row and column position in the answer is the simple product of those positions in A (e.g. row 1, column 2 is the product of position 1 and position 2), and then (2) that the diagonal of the matrix contains the individual products that lead to the scalar answer above. 

What you might also notice is that this is a square symmetric array:


In [252]:
print(B)
print(B.T)
B == B.T


[[ 4 -2 12  6]
 [-2  1 -6 -3]
 [12 -6 36 18]
 [ 6 -3 18  9]]
[[ 4 -2 12  6]
 [-2  1 -6 -3]
 [12 -6 36 18]
 [ 6 -3 18  9]]


array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

Let's extend this to a true but still simple 2-dimensional matrix. Consider the following matrix:

$
A = \begin{bmatrix} 4 &   -2  &  12  \\ -2   &  1  &  -6  \\ 12  &  -6   & 36  \\ -6  &   3  & -18   \end{bmatrix} 
$

We might imagine this matrix reflects a subset of our spatiotemporal data, with 4 years of observations (the row) at 3 locations (the columns).  We know we can find the dot product if the inner dimensions are the same:

In [253]:
A = np.array([[4, -2, 12],[-2, 1, -6],[12, -6, 36],[-6, 3, -18]])
C = np.dot(A.T,A) # at 3x4 matrix times a 4x3 matrix gives us a 3x3 matrix
print(C)

[[ 200 -100  600]
 [-100   50 -300]
 [ 600 -300 1800]]


Once again, notice that the result is a square symmetric matrix:

In [254]:
C==C.T

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

For larger matrices, look at a screen of 'True' or 'False' might not be that efficient (or pleasant), so you could use the `.all()` method appended to the equality to get a single answer:

In [255]:
(C==C.T).all()

True

## Covariance (and correlation) matrices

Finally, let's look at covariance matrices as a special case of a symmetric square matrix that contains information about our spatiotemporal data.  Let's return to our matrix above, but thinking this time specifically about covariance.  

$
A = \begin{bmatrix} 4 &   -2  &  12  \\ -2   &  1  &  -6  \\ 12  &  -6   & 36  \\ -6  &   3  & -18   \end{bmatrix} 
$

We might now imagine this matrix reflects a subset of our spatiotemporal data, with 4 years of observations (the rows) at 3 locations (the columns).  How would we calculate the covariance using just matrix operations?  First, we'll need to remove the mean (since variance is a measure of dispersion around a mean value) and we need to divide by the number of samples less one (for it to be unbiased).  

Let's imagine we want specifically to know the relationship through time of the phenomena we are measuring at each of our locations.  Or, in otherwords, we want to know how much each site goes 'up and down' together through time - are they in sync? Out of sync?  Not related at all? 

Let's start out by removing the mean value from each location (each column, in this matrix).  Remember in the calculation of covariance we are finding the products of the values with the mean subtracted:

$
cov(x,y) = \frac{1}{n-1}\sum_{i}^{n} (x_i - \bar{x})(y_i - \bar{y})
$

To do this efficiently, Python allows us to do something called broadcasting, which describes how `NumPy` treats arrays with different shapes when doing arithmetic operations.  In this case, we can subtract the mean of each column from the data matrix to get the centered or anomaly values:

In [256]:
A_bar = np.mean(A,axis=0)
print(A_bar)


[ 2. -1.  6.]


Notice that to calculate the mean, we had to specific the `axis` in the call to `np.mean` (otherwise, NumPy defaults to flattening the array and just calculating the mean of all the values). 

If we want to remove the mean from the values in each column, we can simply do the following:

In [257]:
print(A - np.mean(A,axis=0)) # subtract the mean of the columns from the respective columns
print(np.mean(A - np.mean(A,axis=0),axis=0)) # check - mean should be zero 

[[  2.  -1.   6.]
 [ -4.   2. -12.]
 [ 10.  -5.  30.]
 [ -8.   4. -24.]]
[0. 0. 0.]


The operation of subtracting a 1x3 array from a 4x3 array works because Python has a set of rules to broadcast unmatched dimensions when performing an operation.  See here for a description of this within NumPy: https://numpy.org/doc/stable/user/basics.broadcasting.html

The next operation we would need to do to calculate covariance using matrix operations would be the matrix multiplication step.  Remember that this part of the covariance equation:

$
\sum_{i}^{n} (x_i - \bar{x})(y_i - \bar{y})
$

Says, take the sum of the products of the corresponding values of vector _x_ and _y_.

Let's see if we can first generative the above equation for the covariance of two vectors to instead indicate the covariance of each column of a data matrix **A** of size $n$ rows and $p$ columns, $a_1$, $a_2$, $a_3$, ... $a_p$:

$
\mathbf{A} = 
\begin{bmatrix}
a_{1,1} & a_{1,2} & \dotsm & a_{1,p}\\
a_{2,1} & a_{2,2} & \dotsm & a_{2,p}\\
\vdots & \vdots & \ddots & \vdots\\
a_{n,1} & a_{n,2} & \dotsm & a_{n,p}
\end{bmatrix}
$

$
cov(a_j,a_k) = \frac{1}{n-1}\sum_{i}^{n} (a_{ij} - \bar{a_j})(a_{ik} - \bar{a_k})
$

Or, the covariance of any two rows $a_j$ and $a_k$ in the matrix **A** is the sum of the products of the elements of each row $n$ with the mean of that column ($j$ or $k$, respectively, removed).  We make recoginize that when $j==k$ we are simply calculating the variance of that column! 

The covariance matrix is a square symmetrical matrix of size _p_ x _p_ describing the covariance of each column of **A** (1 thought _p_) with all the other columns of **A** (1 through _p_) and can be written as:

$
\mathbf{C} = 
\begin{bmatrix}
cov(a_{1},a_{1}) & cov(a_{1},a_{2}) & \dotsm & cov(a_{1},a_{p})\\
cov(a_{2},a_{1}) & cov(a_{2},a_{2}) & \dotsm & cov(a_{2},a_{p})\\
\vdots & \vdots & \ddots & \vdots\\
cov(a_{p},a_{1}) & cov(a_{p},a_{2}) & \dotsm & cov(a_{p},a_{p})
\end{bmatrix}
$

or, since the covariance of one column of **A** with itself is the same as the variance of that column:

$
\mathbf{C} = 
\begin{bmatrix}
var(a_{1}) & cov(a_{1},a_{2}) & \dotsm & cov(a_{1},a_{p})\\
cov(a_{2},a_{1}) & var(a_{2}) & \dotsm & cov(a_{2},a_{p})\\
\vdots & \vdots & \ddots & \vdots\\
cov(a_{p},a_{1}) & cov(a_{p},a_{2}) & \dotsm & var(a_{p})
\end{bmatrix}
$

Observe a few things, some of which are review:  First, the diagonal elements contain the variances of each column.  Next, the matrix will be square, with $p$ rows and $p$ columns.  Third, the matrix is symmetrical (e.g. the value in row 1, column 2 is the same as the value in row 2, column 1).  

What this means is that we have a special matrix (square, symmetrical) that contains all the information about how all the columns of our data matrix relate to one another!

Now, let's finish the matrix operations.  We could go column by column and calculate the covariance of each individually, but we should also be able to see by now that matrix multiplication allows us to do this much quicker!  Since matrix multiplcation the sum of the products of each row of a matrix with the corresponding column in another, if we multiply the transpose of the centered (mean removed) matrix **A_{c}** by the non-tranposed version of the same matrix, we will calculate the sum of the products of each column of the data matrix with itself!  As well, if we take our take the transpose of our $n$ x $p$ matrix -- $p$ x $n$ and multiply it by the original $n$ by $p$ matrix, the inner dimensions (the number of rows in our data matrix) match and the outer dimensions (the number of columns in our data matrix) indicate the matrix product will be $p$ x $p$.  All we need to do is divide by the scale $n-1$ and we've got a covariance matrix:


In [258]:
A0 = A - A.mean(axis=0) # we'll call the centered matrix A0 ('A naught'), it is still 4x3
print(A0)
1/(A0.shape[0]-1) * (A0.T @ A0)

[[  2.  -1.   6.]
 [ -4.   2. -12.]
 [ 10.  -5.  30.]
 [ -8.   4. -24.]]
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.

array([[ 61.33333333, -30.66666667, 184.        ],
       [-30.66666667,  15.33333333, -92.        ],
       [184.        , -92.        , 552.        ]])

Of course, there is an easier way - NumPy offers `np.cov` which can do the matrix calculation for us.  **A warning**, though.  As the `np.cov` documentation says, for the input matrix the functin assumes that eEach row of m represents a variable, and each column a single observation of all those variables.' -- this is the opposite of how we've envisioned our data matrix above and the opposite of how MATLAB, R, and even []`Pandas` in Python](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.cov.html) does it!  

To get the answer we got above, where we calculate the covariance between the columns of our data matrix, and the rows are observations of the variables or locations in each column, we need to invert the centered matrix:

In [259]:
np.cov(A0.T,ddof=1)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


array([[ 61.33333333, -30.66666667, 184.        ],
       [-30.66666667,  15.33333333, -92.        ],
       [184.        , -92.        , 552.        ]])

Note that `np.cov` knows to remove the mean, so we get the same answer if we use the transpose of the data matrix before we removed the mean:

In [260]:
C = np.cov(A.T,ddof=1)
print(C)

[[ 61.33333333 -30.66666667 184.        ]
 [-30.66666667  15.33333333 -92.        ]
 [184.         -92.         552.        ]]
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Just to emphasize my **warning** -- since dimensionality and how different programming languages and libraries and packages make assumptions about dimensionality varies in very important ways, always be extremely careful when calculating covariance matrices (or using any built-in operators) - this is why we've spent some time building a background in these calculations, so that you have an intuition about what to expect and how to identify if something is happening in a different way then you intend.

Let's look at the elements of the covariance matrix above.  It is indeed 3 by 3 and so square. We expect to find that the diagonal elements of the covariance matrix are the variance of the original data columns:

In [261]:
print(np.diagonal(C))
print(np.var(A,axis=0,ddof=1)) # specify to take the variance down columns and us the unbiased estimator 

[ 61.33333333  15.33333333 552.        ]
[ 61.33333333  15.33333333 552.        ]


We should also see that the matrix is symmetrical - the off-diagonal elements are a mirror image of each other.

The covariance matrix encapsulates the similarity (or not) between the columns of the data matrix **A** as well as the total variance in each column.   It seems from the large-ish positive value in row 3, column 1 and in row 1, column 3 of the covariance matrix that column 1 and column 3 of our data matrix are similar, although it is tough to put that number into relative terms - how similar?  Likewise, negative values in both row 2, column 1, and row 1, column 2 as well as row 2, column 3, and row 3, column 2 indicate that the data in column 2  of **A** is negatively covarying with both column 1 and column 3. 

As we've already seen, the correlation matrix is similar to the covariance matrix but is normalized by the variance, meaning correlation coefficients will be between -1 and 1 and therefore more readily interpretable.  Recall the correlation is calculated as: 

$
corr(x,y) = r_{x,y} = \frac{1}{n-1}\sum_{i=1}^n  \Big( \frac{x_i - \bar{x}}{\sigma_x} \Big)  \Big( \frac{y_i - \bar{y}}{\sigma_y} \Big) 
$

Based on how we calculated the covariance matrix above using only matrix operations, we might see that we could do the same for correlation if we normalized (divided by) the variance of each column of the data.

Another way to look at this would be that we can not only remove the mean of each column before calculating their covariance, but also set each column to have the same variance.  

We can do this by dividing by the standard deviation of each column after we remove the mean:

In [262]:
Z = (A - A.mean(axis=0)) / np.std(A,axis=0,ddof=1)
print(np.std(Z,axis=0,ddof=1)) # will reveal that the standard deviation of each column is now 1
print(np.var(Z,axis=0,ddof=1)) # will reveal that the variance of each column is now 1

[1. 1. 1.]
[1. 1. 1.]


Now that we have a matrix **Z** that has column means of 0 and column standard deviations (and variances) of 1, we can do our correlation calculation:

In [263]:
R = 1/(Z.shape[0]-1) * (Z.T @ Z)
print(R)
print(np.corrcoef(Z.T))

[[ 1. -1.  1.]
 [-1.  1. -1.]
 [ 1. -1.  1.]]
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
[[ 1. -1.  1.]
 [-1.  1. -1.]
 [ 1. -1.  1.]]
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Huh!  Look at that.  The correlation matrix indicates that all the columns of the matrix **A** we've been using are perfectly correlation (column 1 and column 3) or perfectly anti-correlation (column 2 with either column 1 or column 2).  So our intuition about their relationship from their covariance was correct, but the correlation matrix more readily reveals exactly how strongly they covaried! 

Let's take another look at that data matrix:

$
A = \begin{bmatrix} 4 &   -2  &  12  \\ -2   &  1  &  -6  \\ 12  &  -6   & 36  \\ -6  &   3  & -18   \end{bmatrix} 
$

With another look, it quickly becomes obvious that our column 2 is just a scaled (by -0.5) version of column 1.  And column 3 is similarly just a scaled (by 3) version of column 1 as well.   Our matrix is strongly linearly dependent!

Remember the `rref()` function in `sympy`?  In addition to finding the 'reduced row echelon form', we can also ask it to return the independent columns of A:

In [264]:
import sympy 
ref, inds = sympy.Matrix(A).T.rref() # transpose because we're looking at the covariance of each column here 
print(ref)
print(inds)


Matrix([[1, -1/2, 3, -3/2], [0, 0, 0, 0], [0, 0, 0, 0]])
(0,)


You'll notice a few things.  In the RREF returned by the code, the 2nd and 3rd rows are all zero, which we learned is an indication of 'free variables' (that $x_2$ and $x_3$ could take any value, or there are infinitely many solutions).  The second output, `inds` tell us that only the 0th column is independent (or, really, that all the columns are a variant of column 1).   

Because this is the case, we might expect to find that the data matrix is rank deficit:

In [265]:
np.linalg.matrix_rank(A) # only rank 1, even though A is 4x3

1

... and that the condition number will not be close to 1 - it may be very large or even infinite:

In [266]:
np.linalg.cond(A)

inf

So our data matrix is rank deficit and ill-conditioned, as we might expect.   However, as we'll soon see, the covariance (or correlation matrix) of a rank deficit, linearly dependent, and ill-conditioned matrix offers a way forward toward dealing with the very strong covariance in our real data systems.